In [ ]:
%run "0a_Workspace_setup.ipynb"
# %run "0b_Create_poi_files.ipynb"

from folium.features import DivIcon
from NHM_helpers.NHM_hydrofabric import * #give list of names, not stars
from NHM_helpers.map_template import * 

In [ ]:
hru_gdf = create_hru_gdf(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    nhru_params,
    nhru_nmonths_params,
)

seg_gdf = create_segment_gdf(
    model_dir,
    GIS_format,
    param_filename,
)

nwis_gages_aoi = fetch_nwis_gage_info(
    model_dir,
    control_file_name,
    nwis_gage_nobs_min,
    hru_gdf,
)

poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    nwis_gages_aoi,
    gages_file,
)

default_gages_file = create_default_gages_file(
    model_dir,
    nwis_gages_aoi,
    poi_df,
)

gages_df = read_gages_file(
    model_dir,
    poi_df,
    nwis_gages_file,
    gages_file,
)

## Set default aspects of maps

### Set approximate latitude, longitude and zoom level for subbasin is calculated for starting point of folium.map plot window.

In [ ]:
pfile_lat, pfile_lon, zoom, cluster_zoom = folium_map_elements(hru_gdf, poi_df, "")

In [ ]:
USGSHydroCached_layer, USGStopo_layer, Esri_WorldImagery, OpenTopoMap = (
    folium_map_tiles()
)

### Set up inset map
This requires folium plugins. (from folium import plugins)

In [ ]:
minimap = create_minimap()

### Create hydrofabric map objects

In [ ]:
hru_map = create_hru_map(hru_gdf)
seg_map_show = create_segment_map_show(seg_gdf)
seg_map_hide = create_segment_map_hide(seg_gdf)

### Set poi (gages) mapping style

In [ ]:
poi_marker_cluster, poi_marker_cluster_label = create_poi_marker_cluster(
    poi_df, cluster_zoom
)

In [ ]:
# isinstance(poi_marker_cluster_label, folium.plugins.MarkerCluster)

### Set non-poi (gages) mapping style

In [ ]:
non_poi_marker_cluster, non_poi_marker_cluster_label = create_non_poi_marker_cluster(
    poi_df, nwis_gages_aoi, gages_df, cluster_zoom
)

## Create interactive HRU map

In [ ]:
from IPython.display import display
from folium.plugins import MeasureControl

m2 = folium.Map()
m2 = folium.Map(
    location=[pfile_lat, pfile_lon],
    tiles=USGSHydroCached_layer,
    zoom_start=zoom,
    width="100%",
    height="100%",
    control_scale=True,
)

USGStopo_layer.add_to(m2)
OpenTopoMap.add_to(m2)
Esri_WorldImagery.add_to(m2)

hru_map.add_to(m2)
seg_map_show.add_to(m2)
seg_map_hide.add_to(
    m2
)  # Tooltip is associated with this layer, and will also always be on top (last mapped).


# Add tool tips to map
# hru_map.add_child(tooltip_hru)
# seg_map_hide.add_child(tooltip_seg)

poi_marker_cluster.add_to(m2)
poi_marker_cluster_label.add_to(m2)


# Add tool tips to map
# hru_map.add_child(tooltip_hru)
# seg_map_hide.add_child(tooltip_seg)

non_poi_marker_cluster.add_to(m2)
non_poi_marker_cluster_label.add_to(m2)

m2.add_child(minimap)
m2.add_child(MeasureControl())

plugins.Fullscreen(position="bottomleft").add_to(m2)
folium.LayerControl(collapsed=True, position="bottomright").add_to(m2)

##add Non-poi gage markers and labels using row df.interowss loop
gages_list = gages_df.index.to_list()
additional_gages = list(set(gages_list) - set(poi_df.poi_id))

# display(m2)

con.print(
    f"[underline]The {subbasin} Model Hydrofabric[/underline]\n",
    f"\n{pdb.dimensions.get('nhru').meta['description']} (nhru), {pdb.dimensions.get('nhru').meta['size']}",
    f"\n{pdb.dimensions.get('nsegment').meta['description']} (nsegment), {pdb.dimensions.get('nsegment').meta['size']}",
    f"\n{pdb.dimensions.get('npoigages').meta['description']} (npoigages), {pdb.dimensions.get('npoigages').meta['size']}",
    f"\nNumber of Non-poi gages, {len(additional_gages)}",
)

display(m2)